<a href="https://colab.research.google.com/github/IllgamhoDuck/Quantum-Circuit-Optimization-with-Deep-learning/blob/master/layout_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM encoder layout prediction
**Estimating layout with LSTM encoder with noise and hardware topology**

### Setting environment
- package install
- google drive mount
- path configuration

In [0]:
 !pip install --quiet qiskit nxpd

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# TODO - modify path to fit your computer
# dataset_path - where to save the dataset including circuit information
# model_path = where to save the deep learning model you created
# weight_path = Where to save the model weight you trained

dataset_path = '/content/drive/My Drive/QUANTUM/qaward/dataset'
model_path = '/content/drive/My Drive/QUANTUM/qaward/model and weight'
weight_path = '/content/drive/My Drive/QUANTUM/qaward/model and weight'

### Train step 0 - load the total dataset
- Load dataset
- Divide it to Train / Validation / Test (default 70% / 10% / 20%)
- check the shape of data

In [0]:
import os
import pickle

# TODO - Choose your dataset name
dataset_name = "5_qubit_encoded_dataset.pkl"
with open(os.path.join(dataset_path, dataset_name), 'rb') as f:
    total_dataset = pickle.load(f)

**Store dataset as a global variable**

In [0]:
for k, v in total_dataset.items():
    print(k)
    exec("{} = v".format(k))

enc_in
enc_out_type
enc_out_gate
enc_out_control
enc_out_target
enc_out_classic
enc_out_angle
dec_in
dec_out_concatenated
dec_out_type
dec_out_gate
dec_out_control
dec_out_target
dec_out_classic
dec_out_angle
layout_out
hardware_in
coupling_map
u1_error
u2_error
u3_error
cx_error
readout_error


**Check is layout data is valid. it should be not fixed to one type**

It should be diverse type like following example

    [[1.   0.25 0.5  0.   0.75]
     [1.   0.25 0.5  0.   0.75]
    [0.   0.25 0.75 0.5  1.  ]
    [1.   0.25 0.5  0.   0.75]
    [0.75 0.25 0.   0.5  1.  ]
    [0.75 0.25 0.   0.5  1.  ]
    [0.25 1.   0.75 0.   0.5 ]
    [0.25 1.   0.   0.5  0.75]
    [0.   0.25 0.5  0.75 1.  ]
    [0.5  0.25 0.75 1.   0.  ]]

In [0]:
print(layout_out[:10])

[[1.   0.25 0.5  0.   0.75]
 [1.   0.25 0.5  0.   0.75]
 [0.   0.25 0.75 0.5  1.  ]
 [1.   0.25 0.5  0.   0.75]
 [0.75 0.25 0.   0.5  1.  ]
 [0.75 0.25 0.   0.5  1.  ]
 [0.25 1.   0.75 0.   0.5 ]
 [0.25 1.   0.   0.5  0.75]
 [0.   0.25 0.5  0.75 1.  ]
 [0.5  0.25 0.75 1.   0.  ]]


**Devide dataset to `Train / Validation / Test` set**

    Train - 70%
    Validation - 10%
    Test - 20%

In [0]:
import numpy as np

data_len = enc_in.shape[0]
# TODO - choose the ratio
# Test ratio will be choosed by train and validation ratio
# 0.7 means 70% if dataset is 10,000 it will be 7,000
train_ratio = 0.7
validation_ratio = 0.1

# this ratio will be used on the below code
# first ratio - fr / second ratio - sr
fr = train_ratio
sr = train_ratio + validation_ratio

# Generate input
train_enc_in = enc_in[:round(data_len*fr)]
validation_enc_in = enc_in[round(data_len*fr):round(data_len*sr)]
test_enc_in = enc_in[round(data_len*sr):]

# Generate hareware information -> Noise and topologie input
train_hardware_in = hardware_in[:round(data_len*fr)]
validation_hardware_in = hardware_in[round(data_len*fr):round(data_len*sr)]
test_hardware_in = hardware_in[round(data_len*sr):]

# Generate output
train_layout_out = layout_out[:round(data_len*fr)]
validation_layout_out = layout_out[round(data_len*fr):round(data_len*sr)]
test_layout_out = layout_out[round(data_len*sr):]

In [0]:
print(train_hardware_in.shape)

(21000, 70)


### Train step 1 - Create Model
- Choose model hyperparamter
- Create lstm encoder layout prediction model
- check the model

In [0]:
import numpy as np
import keras
from keras import layers
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Input, LSTM, RepeatVector, LeakyReLU, ReLU, BatchNormalization
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta
from keras import objectives
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json

In [0]:
# Model hyperparmeter
# TODO - choose the latent dimension
# This indicates the circuit latent space (manifold)
latent_dim = 32

circuit_dim = enc_in.shape[2]
timesteps = enc_in.shape[1]
env_dim = hardware_in.shape[1]
layout_n = layout_out.shape[1]

In [0]:
def layout_model(circuit_dim, timesteps, env_dim, latent_dim, layout_n):
    # Encode the circuit to manifold
    circuit_input = Input(shape=(timesteps, circuit_dim), name='circuit_encode')
    encoded = LSTM(64)(circuit_input)
    # encoded = Dense(64)(encoded)
    # encoded = LeakyReLU(alpha=0.05)(encoded)
    # encoded = Dense(64)(encoded)
    # encoded = LeakyReLU(alpha=0.05)(encoded)

    # z = Dense(latent_dim)(encoded)
    # z = LeakyReLU(alpha=0.05)(z)

    # (batch_size, 1, circuit_dim) -> (batch_size, circuit_dim)
    # z = Flatten()(z)

    # Hardware information include hardware topologie and noise information
    hardware_input = Input(shape=(env_dim,), name='hardware')

    # Concatenate z and hardware information
    concatenated = layers.concatenate([encoded, hardware_input])

    # Dense layer
    x = Dense(128)(concatenated)
    x = Dense(128, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    x = Dense(32)(x)
    x = BatchNormalization()(x)

    # Estimated layout
    layout = Dense(layout_n, activation='sigmoid')(x)

    layout_model = Model([circuit_input, hardware_input], layout)

    layout_model.compile(optimizer=Adam(0.001),
                             loss='mse',
                             metrics=['accuracy'])
    return layout_model

In [0]:
model = layout_model(circuit_dim, timesteps, env_dim, latent_dim, layout_n)
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
circuit_encode (InputLayer)     (None, 65, 29)       0                                            
__________________________________________________________________________________________________
lstm_7 (LSTM)                   (None, 64)           24064       circuit_encode[0][0]             
__________________________________________________________________________________________________
hardware (InputLayer)           (None, 70)           0                                            
__________________________________________________________________________________________________
concatenate_7 (Concatenate)     (None, 134)          0           lstm_7[0][0]                     
                                                                 hardware[0][0]             

### Train step 2 - Train the model
- Choose train hyperparameter
    - epochs
    - batch_size
    - shuffle
- Save the model and weight
- Load the weight for sequential train
- Load the weight for transfer learning from lstm autoencoder

#### Load the weight for sequential train
**Model should be already pre-built**

In [0]:
# TODO - Choose weight name to load
model_weight = "layout.h5"

model.load_weights(os.path.join(weight_path, model_weight))
print("Loaded weight for sequential model")

Loaded weight for sequential model


#### Transfer learning from lstm autoencoder
**Loading the weight of lstm encoder part**

In [0]:
# TODO - Choose weight name to load
enc_weight = "lstm_ae_enc.h5"

model.load_weights(os.path.join(weight_path, enc_weight))
print("Loaded weight for transfering learning")

#### Train

In [0]:
# Train hyperparameter
# TODO - Choose the hyperparameter
shuffle = True
batch_size = 256
epochs = 100

model.fit([train_enc_in, train_hardware_in],
          train_layout_out,
          verbose=1,
          shuffle=shuffle,
          epochs=epochs,
          batch_size=batch_size,  
          validation_data=([validation_enc_in, validation_hardware_in],
                           validation_layout_out))

**Save model and weights**

In [0]:
model_json = model.to_json()
with open(os.path.join(model_path, "layout.json"), "w") as f:
    f.write(model_json)
print("Saved model to disk")    
model.save_weights(os.path.join(weight_path, "layout.h5"))
print("Saved weights to disk")

Saved model to disk
Saved weight to disk


### Train step 3 - Generate the circuit and compare it with the original circuit

In [0]:
#@title Compare layout function { display-mode: "form" }
import copy
def compare_layout(original_layout, predicted_layout):
    # Get the layout of original
    original_layout_order = copy.deepcopy(original_layout)
    original_layout_order.sort()
    original_layout = [np.where(original_layout_order == original_layout[i])[0][0] for i in range(len(original_layout))]
    
    # Get the layout of predicted
    predicted_layout_order = copy.deepcopy(predicted_layout)
    predicted_layout_order.sort()
    predicted_layout = [np.where(predicted_layout_order == predicted_layout[i])[0][0] for i in range(len(predicted_layout))]
    
    # Is original layout and predicted layout same?
    is_same = original_layout == predicted_layout
    return is_same, original_layout, predicted_layout

In [0]:
#@title Comparing the original and predicted layout results
layout_result = "is_same?, original_layout, original_encode, predicted_layout, predicted_encode\n"
total = 0
correct = 0
num = 0
for enc, env, original_layout in zip(test_enc_in, test_hardware_in, test_layout_out):
    predicted_layout = model.predict([[enc], [env]])[0]
    result, o_layout, p_layout = compare_layout(original_layout, predicted_layout)
    layout_result += str(result) + ', ' + str(o_layout) + ', ' + str(original_layout)
    layout_result += ', ' + str(p_layout) + ', ' + str(predicted_layout) + '\n'
    total += 1
    if (result):
        correct += 1
print("Accuracy {}% ({} / {})".format(round(correct / total * 100, 2), correct, total))

Accuracy 60.35% (3621 / 6000)


In [0]:
print(layout_result)

is_same?, original_layout, original_encode, predicted_layout, predicted_encode
False, [4, 3, 0, 2, 1], [1.   0.75 0.   0.5  0.25], [4, 1, 3, 2, 0], [0.95161545 0.21475577 0.7588526  0.24665087 0.18329133]
False, [1, 2, 4, 3, 0], [0.25 0.5  1.   0.75 0.  ], [1, 2, 3, 4, 0], [0.29971606 0.46958625 0.82536566 0.94859076 0.02785778]
False, [3, 1, 2, 4, 0], [0.75 0.25 0.5  1.   0.  ], [4, 1, 2, 0, 3], [0.9145075  0.3002935  0.3353102  0.14989883 0.7521246 ]
False, [1, 0, 2, 3, 4], [0.25 0.   0.5  0.75 1.  ], [1, 3, 2, 0, 4], [0.22279032 0.8029408  0.507625   0.07366765 0.9172792 ]
False, [2, 0, 1, 4, 3], [0.5  0.   0.25 1.   0.75], [1, 0, 2, 3, 4], [0.39102787 0.07775862 0.4268181  0.83328414 0.8451133 ]
True, [1, 0, 3, 2, 4], [0.25 0.   0.75 0.5  1.  ], [1, 0, 3, 2, 4], [0.29685423 0.03855658 0.8633811  0.48475736 0.9101771 ]
True, [3, 0, 4, 1, 2], [0.75 0.   1.   0.25 0.5 ], [3, 0, 4, 1, 2], [0.84117764 0.06953368 0.92520136 0.27401167 0.3650723 ]
False, [0, 4, 1, 2, 3], [0.   1.   0.25 0